In [21]:
from typing import List
from dataclasses import fields
import modelspec
from modelspec import field, instance_of, optional
from modelspec.base_types import Base
from typing import List
from modelspec.utils import save_to_xml_file
import xml.dom.minidom

In [8]:
@modelspec.define
class Population(Base):
    """
    Some description...

    Args:
        id: The id of the population
        component: the component to use in the population
        size: the size of the population
    """

    id: str = field(validator=instance_of(str))
    component: str = field(default=None, validator=optional(instance_of(str)))
    size: int = field(default=None, validator=optional(instance_of(int)))


@modelspec.define
class ExplicitInput(Base):
    """
    Some description...

    Args:
        target: the target of the input
        input: the input, e.g. pulseGenerator
    """

    target: str = field(default=None, validator=optional(instance_of(str)))
    input: str = field(default=None, validator=optional(instance_of(str)))


@modelspec.define
class Network(Base):
    """
    Some description...

    Args:
        id: The id of the network
        populations: the pops in the net
    """

    id: str = field(validator=instance_of(str))

    populations: List[Population] = field(factory=list)
    explicitInputs: List[ExplicitInput] = field(factory=list)


@modelspec.define
class PulseGenerator(Base):
    """
    Some description...

    Args:
        id: The id of the pulseGenerator
        delay: the delay
        duration: the duration
        amplitude: the amplitude
    """

    id: str = field(validator=instance_of(str))
    delay: str = field(validator=instance_of(str))
    duration: str = field(validator=instance_of(str))
    amplitude: str = field(validator=instance_of(str))


@modelspec.define
class Izhikevich2007Cell(Base):
    """
    Some description...

    Args:
        id: The id of the cell...
    """

    id: str = field(validator=instance_of(str))

    C: str = field(validator=instance_of(str))
    v0: str = field(validator=instance_of(str))
    k: str = field(validator=instance_of(str))
    vr: str = field(validator=instance_of(str))
    vt: str = field(validator=instance_of(str))
    vpeak: str = field(validator=instance_of(str))
    a: str = field(validator=instance_of(str))
    b: str = field(validator=instance_of(str))
    c: str = field(validator=instance_of(str))
    d: str = field(validator=instance_of(str))


@modelspec.define
class NeuroML(Base):
    """
    Some description...

    Args:
        id: The id of the NeuroML 2 document
        version: NeuroML version used
        networks: The networks present
    """

    id: str = field(validator=instance_of(str))
    version: str = field(validator=instance_of(str))

    izhikevich2007Cells: List[Izhikevich2007Cell] = field(factory=list)
    pulseGenerators: List[PulseGenerator] = field(factory=list)
    networks: List[Network] = field(factory=list)


nml_doc = NeuroML(id="TestNeuroML", version="NeuroML_v2.3")

izh = Izhikevich2007Cell(
        id="izh2007RS0",
        C="100pF",
        v0="-60mV",
        k="0.7nS_per_mV",
        vr="-60mV",
        vt="-40mV",
        vpeak="35mV",
        a="0.03per_ms",
        b="-2nS",
        c="-50.0mV",
        d="100pA",
)
nml_doc.izhikevich2007Cells.append(izh)

pg = PulseGenerator(
        id="pulseGen_0", delay="100ms", duration="800ms", amplitude="0.07 nA"
)
nml_doc.pulseGenerators.append(pg)

net = Network(id="IzNet")
nml_doc.networks.append(net)

net.populations.append(Population("IzhPop0", component="izh2007RS0", size=1))
net.explicitInputs.append(ExplicitInput(target="IzhPop0[0]", input="pulseGen_0"))
nml_dict = nml_doc.to_dict()

In [26]:
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom

def convert_to_xml(obj):
    root = ET.Element(obj.__class__.__name__)
    add_attributes(obj, root)
    tree = ET.ElementTree(root)
    xml_data = ET.tostring(root, encoding='utf-8')
    parsed_xml = minidom.parseString(xml_data)
    return parsed_xml.toprettyxml(indent="  ")

def add_attributes(obj, parent_element):
    for attr, value in vars(obj).items():
        if isinstance(value, list):
            for item in value:
                if hasattr(item, '__dict__'):
                    child_element = ET.SubElement(parent_element, attr)
                    add_attributes(item, child_element)
                else:
                    ET.SubElement(parent_element, attr).text = str(item)
        elif isinstance(value, dict):
            for sub_attr, sub_value in value.items():
                if hasattr(sub_value, '__dict__'):
                    child_element = ET.SubElement(parent_element, attr)
                    sub_element = ET.SubElement(child_element, sub_attr)
                    add_attributes(sub_value, sub_element)
                else:
                    ET.SubElement(parent_element, attr, {sub_attr: str(sub_value)})
        elif hasattr(value, '__dict__'):
            child_element = ET.SubElement(parent_element, attr)
            add_attributes(value, child_element)
        else:
            ET.SubElement(parent_element, attr).text = str(value)


print(convert_to_xml(nml_doc))

TypeError: vars() argument must have __dict__ attribute

In [13]:
def build_xml_element(obj):
    for attr, value in vars(obj).items():
        print("attr: {} value{}".format(attr, value))
    
build_xml_element(nml_doc)

TypeError: vars() argument must have __dict__ attribute

In [25]:
def loop_through_class(obj):
    attributes = vars(obj)

    for attr_name, attr_value in attributes.items():
        print(f"Attribute: {attr_name} = {attr_value}")

        if isinstance(attr_value, type):
            loop_through_class(attr_value)

loop_through_class(nml_doc)

TypeError: vars() argument must have __dict__ attribute

In [16]:
dir(nml_doc)

['__annotations__',
 '__attrs_attrs__',
 '__attrs_own_setattr__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cls_generate_documentation',
 '_is_base_type',
 '_is_child_field',
 '_is_evaluable_expression',
 '_parse_allowed_children',
 '_parse_allowed_fields',
 '_parse_definition',
 '_type_to_str',
 'allowed_children',
 'allowed_fields',
 'definition',
 'from_bson',
 'from_bson_file',
 'from_dict',
 'from_file',
 'from_json',
 'from_json_file',
 'from_xml',
 'from_xml_file',
 'from_yaml_file',
 'generate_documentation',
 'get_child',
 'id',
 'izhikevich2007Cells',
 'networks',
 'pulseGenerators',
 'to_bson',
 '

In [ ]:
def to_xml(obj: Any) -> Any:
    """ Convert a class object and its members to XML.

    Each class member is treated as a tag to the current XML-element.
    Each member object is treated as a new sub-element.
    Each 'list' member is treated as a new list tag.
    """
 
    if isinstance(obj, dict):
        raise Exception("Dictionary type is not supported.")

    root = None
    tags = {}

    subelements = {} # type: Dict[Any, Any]
    for member in get_members(obj):
        item = getattr(obj, member)
        member = member.replace('_', '-')

        # if object is None, add empty tag
        if item is None:
            subelements[member] = et.Element(member)
        else:
            if not isinstance(item, (str, XmlElement, list, set, tuple)):
                raise Exception("Attributes must be an expected type, but was: {}".format(type(item)))
 

            # Add list sub-elements
            if isinstance(item, (list, set, tuple)):
                subelements[member] = []
                for list_object in item:
                    subelements[member].append(to_xml(list_object))
            # Add sub-element
            elif isinstance(item, XmlElement):
                subelements[member] = to_xml(item)
            # Add element's tag name
            else:
                tags[member] = item

    try:
        if obj._NAME:
            root = et.Element(obj._NAME, tags)
        else:
            raise Exception("Name attribute can't be empty.")
    except (AttributeError, TypeError) as ex:
        print("Attribute value or type is wrong. %s: %s", obj, ex)
        raise

    # Add sub elements if any
    if subelements:
        for name, values in subelements.items():
            if isinstance(values, list): # if list of elements. Add all sub-elements
                sub = et.SubElement(root, name)
                for value in values:
                    sub.append(value)
            else: # single sub-child or None
                if values is None: # if None, add empty tag with name
                    sub = et.SubElement(root, name)
                else: # else add object
                    root.append(values)

    try:
        if obj._TEXT:
            root.text = obj._TEXT
    except AttributeError as ex:
        print("Attribute does not exists. %s: %s", obj, ex)
        raise

    return root

In [19]:
for attr, value in vars(nml_doc).items():
    print("{} {}".format(attr, value))

TypeError: vars() argument must have __dict__ attribute

In [22]:
def build_xml_element(parent, data):
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, dict):
                element = ET.SubElement(parent, key.replace(" ", "_"))#.set(key[1:], str(value))
                #build_xml_element(element, value)
            elif isinstance(value, list):
                for item in value:
                    subelement = ET.SubElement(parent, key.replace(" ", "_"))#.set(key[1:], str(value))
                    build_xml_element(subelement, item)
                    #subelement.set(key, str(value))
            else:
                parent.set(key, str(value))
                #element = ET.SubElement(parent, key.replace(" ", "_"))#.set(key[1:], str(value))
                #element.set(key, str(value))
                #element.text = str(value)
    else:
        parent.text = str(data)
        

        
root = ET.Element("root")
build_xml_element(root, nml_dict)
xml_string = ET.tostring(root).decode()
dom = xml.dom.minidom.parseString(xml_string)
pretty_xml = dom.toprettyxml(indent="  ")
print(pretty_xml)

<?xml version="1.0" ?>
<root>
  <TestNeuroML/>
</root>



In [24]:
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom

def object_to_xml(obj, root_name='root'):
    root = ET.Element(root_name)

    def _dict_to_xml(element, dictionary):
        for key, value in dictionary.items():
            if isinstance(value, dict):
                _dict_to_xml(element, value)
            elif isinstance(value, list):
                for item in value:
                    child = ET.Element(key)
                    _dict_to_xml(child, item)
                    element.append(child)
            else:
                element.set(key, str(value))

    if isinstance(obj, dict):
        _dict_to_xml(root, obj)
        
    xml_string = ET.tostring(root, encoding='utf-8')
    parsed_xml = minidom.parseString(xml_string)
    pretty_xml = parsed_xml.toprettyxml(indent="  ")

    return pretty_xml

xml_data = object_to_xml(nml_dict, 'neuroml')
print(xml_data)


<?xml version="1.0" ?>
<neuroml version="NeuroML_v2.3" C="100pF" v0="-60mV" k="0.7nS_per_mV" vr="-60mV" vt="-40mV" vpeak="35mV" a="0.03per_ms" b="-2nS" c="-50.0mV" d="100pA" delay="100ms" duration="800ms" amplitude="0.07 nA" component="izh2007RS0" size="1">
  <explicitInputs target="IzhPop0[0]" input="pulseGen_0"/>
</neuroml>



In [31]:
def build_xml_element(parent, data):
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, dict):
                element = ET.SubElement(parent, key.replace(" ", "_"))#.set(key[1:], str(value))
                build_xml_element(element, value)
            elif isinstance(value, list):
                for item in value:
                    subelement = ET.SubElement(parent, key.replace(" ", "_"))#.set(key[1:], str(value))
                    build_xml_element(subelement, item)
                    #subelement.set(key, str(value))
            else:
                parent.set(key, str(value))
                #element = ET.SubElement(parent, key.replace(" ", "_"))#.set(key[1:], str(value))
                #element.set(key, str(value))
                #element.text = str(value)
    else:
        parent.text = str(data)
        
        
        
root = ET.Element("root")
build_xml_element(root, nml_dict)
xml_string = ET.tostring(root).decode()
dom = xml.dom.minidom.parseString(xml_string)
pretty_xml = dom.toprettyxml(indent="  ")
print(pretty_xml)

<?xml version="1.0" ?>
<root>
  <TestNeuroML version="NeuroML_v2.3">
    <izhikevich2007Cells>
      <izh2007RS0 C="100pF" v0="-60mV" k="0.7nS_per_mV" vr="-60mV" vt="-40mV" vpeak="35mV" a="0.03per_ms" b="-2nS" c="-50.0mV" d="100pA"/>
    </izhikevich2007Cells>
    <pulseGenerators>
      <pulseGen_0 delay="100ms" duration="800ms" amplitude="0.07 nA"/>
    </pulseGenerators>
    <networks>
      <IzNet>
        <populations>
          <IzhPop0 component="izh2007RS0" size="1"/>
        </populations>
        <explicitInputs target="IzhPop0[0]" input="pulseGen_0"/>
      </IzNet>
    </networks>
  </TestNeuroML>
</root>



In [33]:
def get_members(obj: object) -> List[str]:
    """Helper function to get class members other than functions and private members."""
    excluded_members = ['allowed_children', 'allowed_fields', 'definition']
    members = [attr for attr in dir(obj) if not callable(getattr(obj, attr))
               and not attr.startswith('_') and attr not in excluded_members]
    return members

for i in get_members(nml_doc):
    attrib = getattr(nml_doc, str(i))
    print("=="*50)
    print(attrib)

TestNeuroML
[Izhikevich2007Cell(id='izh2007RS0', C='100pF', v0='-60mV', k='0.7nS_per_mV', vr='-60mV', vt='-40mV', vpeak='35mV', a='0.03per_ms', b='-2nS', c='-50.0mV', d='100pA')]
[Network(id='IzNet', populations=[Population(id='IzhPop0', component='izh2007RS0', size=1)], explicitInputs=[ExplicitInput(target='IzhPop0[0]', input='pulseGen_0')])]
[PulseGenerator(id='pulseGen_0', delay='100ms', duration='800ms', amplitude='0.07 nA')]
NeuroML_v2.3


In [34]:
import xml.etree.ElementTree as ET
import xml.dom.minidom
def build_xml_element(parent, data):
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, dict):
                element = ET.SubElement(parent, key.replace(" ", "_"))
                build_xml_element(element, value)
            elif isinstance(value, list):
                for item in value:
                    subelement = ET.SubElement(parent, key.replace(" ", "_"))
                    build_xml_element(subelement, item)
            else:
                element = ET.SubElement(parent, key.replace(" ", "_"))
                element.text = str(value)
    else:
        parent.text = str(data)

root = ET.Element("root")
build_xml_element(root, nml_doc)
xml_string = ET.tostring(
            root, encoding="utf-8", xml_declaration=False, method="xml"
        ).decode("utf-8")

parsed_xml = xml.dom.minidom.parseString(xml_string)
pretty_xml = parsed_xml.toprettyxml(indent=" " * 4)
print(pretty_xml)

<?xml version="1.0" ?>
<root>NeuroML(id='TestNeuroML', version='NeuroML_v2.3', izhikevich2007Cells=[Izhikevich2007Cell(id='izh2007RS0', C='100pF', v0='-60mV', k='0.7nS_per_mV', vr='-60mV', vt='-40mV', vpeak='35mV', a='0.03per_ms', b='-2nS', c='-50.0mV', d='100pA')], pulseGenerators=[PulseGenerator(id='pulseGen_0', delay='100ms', duration='800ms', amplitude='0.07 nA')], networks=[Network(id='IzNet', populations=[Population(id='IzhPop0', component='izh2007RS0', size=1)], explicitInputs=[ExplicitInput(target='IzhPop0[0]', input='pulseGen_0')])])</root>

